<a href="https://colab.research.google.com/github/sagarsahu27/tflite_notebook/blob/main/ResNet50Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.3.0
Eager mode:  True
Hub version:  0.9.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


In [4]:

model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(dtype=tf.float32, input_shape=([224,224,3])))
model.add(hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/classification/1"))
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
with tf.io.gfile.GFile(os.path.join("model.tflite"), 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpxvslkxin/assets


INFO:tensorflow:Assets written to: /tmp/tmpxvslkxin/assets


In [6]:
%ls -alh

total 98M
drwxr-xr-x 1 root root 4.0K Oct 15 14:16 ./
drwxr-xr-x 1 root root 4.0K Oct 15 14:07 ../
drwxr-xr-x 1 root root 4.0K Oct  5 16:31 .config/
-rw-r--r-- 1 root root  98M Oct 15 14:16 model.tflite
drwxr-xr-x 1 root root 4.0K Oct  5 16:31 sample_data/


In [7]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quant = converter.convert()
with tf.io.gfile.GFile(os.path.join("model_quant.tflite"), 'wb') as f:
    f.write(tflite_model_quant)

INFO:tensorflow:Assets written to: /tmp/tmpsxkq_ir2/assets


INFO:tensorflow:Assets written to: /tmp/tmpsxkq_ir2/assets


In [8]:
%ls -alh

total 122M
drwxr-xr-x 1 root root 4.0K Oct 15 14:24 ./
drwxr-xr-x 1 root root 4.0K Oct 15 14:07 ../
drwxr-xr-x 1 root root 4.0K Oct  5 16:31 .config/
-rw-r--r-- 1 root root  25M Oct 15 14:24 model_quant.tflite
-rw-r--r-- 1 root root  98M Oct 15 14:16 model.tflite
drwxr-xr-x 1 root root 4.0K Oct  5 16:31 sample_data/


In [17]:
IMAGE_SIZE = 224
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.tgz", 
                                   extract=True)

flowers_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

# A generator that provides a representative dataset
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files(flowers_dir + '/*/*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]



In [18]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input and output tensors to uint8 (added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

In [19]:
converter.experimental_new_converter = True
tflite_model_quant_int = converter.convert()
with tf.io.gfile.GFile(os.path.join("model_quant_int.tflite"), 'wb') as f:
    f.write(tflite_model_quant_int)
%ls -alh

INFO:tensorflow:Assets written to: /tmp/tmp6p_0_xu4/assets


INFO:tensorflow:Assets written to: /tmp/tmp6p_0_xu4/assets


total 148M
drwxr-xr-x 1 root root 4.0K Oct 15 14:55 ./
drwxr-xr-x 1 root root 4.0K Oct 15 14:07 ../
drwxr-xr-x 1 root root 4.0K Oct  5 16:31 .config/
-rw-r--r-- 1 root root  26M Oct 15 14:55 model_quant_int.tflite
-rw-r--r-- 1 root root  25M Oct 15 14:24 model_quant.tflite
-rw-r--r-- 1 root root  98M Oct 15 14:16 model.tflite
drwxr-xr-x 1 root root 4.0K Oct  5 16:31 sample_data/


In [20]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

interpreter = tf.lite.Interpreter(model_content=tflite_model_quant_int)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.float32'>
output:  <class 'numpy.float32'>
input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>
